# 学習させたVisionTransformerモデルをテストデータで再度推論させる

In [1]:
import os
import sys
sys.path.append('../tools')
sys.path.append("../models")
from model import ModelClass
from _make_dataloader import _make_dataloader
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import onnx
import onnxruntime as ort
import numpy as np

In [2]:
# archive内の画像を前処理して、データローダーを作成
train_loader, val_loader, test_loader = _make_dataloader()

画像の読み込み完了
0〜5000の範囲でバッチ処理完了
5001〜10000の範囲でバッチ処理完了
10001〜15000の範囲でバッチ処理完了
15001〜の範囲でバッチ処理完了
画像の前処理完了
DataLoader作成完了
train_loader:  547
val_loader:  97
test_loader:  114


In [3]:
# モデルのインスタンス化
model = ModelClass()

model.load_state_dict(torch.load('../models/vit_cls.pth'))

# モデルをデバイスに移動
device = torch.device('mps')
model.to(device)

# モデルの評価
model.eval()

batch_size = 32
correct = 0
total = 0

with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        
        # 予測ラベル取得
        _, predicted = torch.max(outputs, 1)
        
        # 正解数のカウント
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        print(f'予測ラベル：{predicted[:10]}')
        print(f'正解ラベル：{labels[:10]}')
        print('----------------')
        i += 1
        if i > 5 :
            break
    
# 精度計算
accuracy = 100 * correct/total

print(f"Test Accuracy: {accuracy:.2f}%")

予測ラベル：tensor([9, 6, 4, 9, 8, 0, 8, 3, 0, 8], device='mps:0')
正解ラベル：tensor([9, 6, 4, 9, 8, 0, 8, 3, 0, 8], device='mps:0')
----------------
予測ラベル：tensor([1, 1, 8, 9, 8, 5, 4, 5, 0, 2], device='mps:0')
正解ラベル：tensor([1, 1, 8, 9, 8, 5, 4, 0, 0, 2], device='mps:0')
----------------
予測ラベル：tensor([7, 9, 9, 9, 4, 0, 0, 0, 4, 9], device='mps:0')
正解ラベル：tensor([7, 9, 9, 9, 4, 0, 0, 0, 4, 9], device='mps:0')
----------------
予測ラベル：tensor([0, 6, 9, 3, 4, 5, 4, 0, 2, 6], device='mps:0')
正解ラベル：tensor([0, 6, 9, 3, 4, 5, 4, 0, 2, 6], device='mps:0')
----------------
予測ラベル：tensor([4, 7, 0, 1, 2, 0, 9, 6, 9, 4], device='mps:0')
正解ラベル：tensor([4, 7, 0, 1, 2, 0, 9, 6, 9, 4], device='mps:0')
----------------
予測ラベル：tensor([6, 4, 4, 9, 4, 0, 5, 3, 3, 0], device='mps:0')
正解ラベル：tensor([6, 4, 4, 9, 4, 0, 5, 3, 3, 0], device='mps:0')
----------------
Test Accuracy: 97.92%


# 変換したONNX形式のモデルで再度推論

onnx-optimizer-tool > convert_to_onnx.pyを使用して変換した`.onnx`ファイルを使う  
`vit_cls.pth`にtest_loaderを推論させたときの精度を比較する。（learn_vit.ipynbを参照）

## ONNXモデルで推論させる

In [4]:
# モデルのロード
onnx_path = "../models/vit_cls.onnx"

onnx_model = onnx.load(onnx_path)

# モデルの検証
onnx.checker.check_model(onnx_model)
print("ONNXモデルの検証が成功しました！")


# ONNXRuntimeセッションを作成
ort_session = ort.InferenceSession(onnx_path)

# 出力名を確認
output_name = [output.name for output in ort_session.get_outputs()]

# 推論を実行

correct = 0
total = 0

for i, (images, labels) in enumerate(test_loader):
	ort_inputs = {ort_session.get_inputs()[0].name: images.numpy()}
	ort_outputs = ort_session.run(output_name, ort_inputs)
	predicted = np.argmax(ort_outputs[0], axis=1)
	labels = labels.numpy()
	# 正解数のカウント
	total += len(labels)
	correct += (predicted == labels).sum()
	
	print(f'予測ラベル：{predicted[:10]}')
	print(f'正解ラベル：{labels[:10]}')
	print('----------------')
	i += 1
	if i > 5 :
		break


# 精度計算
accuracy = 100 * correct/total

print(f"Test Accuracy: {accuracy:.2f}%")

ONNXモデルの検証が成功しました！
予測ラベル：[9 6 4 9 8 0 8 3 0 8]
正解ラベル：[9 6 4 9 8 0 8 3 0 8]
----------------
予測ラベル：[1 1 8 9 8 5 4 5 0 2]
正解ラベル：[1 1 8 9 8 5 4 0 0 2]
----------------
予測ラベル：[7 9 9 9 4 0 0 0 4 9]
正解ラベル：[7 9 9 9 4 0 0 0 4 9]
----------------
予測ラベル：[0 6 9 3 4 5 4 0 2 6]
正解ラベル：[0 6 9 3 4 5 4 0 2 6]
----------------
予測ラベル：[4 7 0 1 2 0 9 6 9 4]
正解ラベル：[4 7 0 1 2 0 9 6 9 4]
----------------
予測ラベル：[6 4 4 9 4 0 5 3 3 0]
正解ラベル：[6 4 4 9 4 0 5 3 3 0]
----------------
Test Accuracy: 97.92%


## Netronで可視化

In [ ]:
import netron

# Visualize the ONNX model
netron.start(onnx_path)

**結論：変換前後で精度は変わらなかった**